In [25]:
import torch
import random

In [41]:
# Define the original tensor
#original_tensor = torch.tensor([1, 2, 3])

# Define the size
class_num = 5
size = 3
batch_size = 5

label = torch.randint(0, class_num-1, (batch_size,))
original_tensor = torch.randint(0, 10, (batch_size, 3))

# Create a new tensor with each element repeated 'size' times
repeated_tensor = original_tensor.repeat_interleave(size).reshape(batch_size, size, -1)
print(label)
#print(repeated_tensor)
#repeated_tensor = repeated_tensor.reshape(class_num, size, 1)

label_idx = list(range(len(label)*size))
random.shuffle(label_idx)
#label_shuffle = torch.tensor(label[label_idx])
print(label_idx)



tensor([2, 2, 1, 3, 1])
[0, 8, 2, 13, 1, 4, 9, 5, 11, 3, 12, 10, 6, 14, 7]


In [44]:
import torch
import random
from itertools import groupby

# 元のテンソル
original_tensor = torch.tensor([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 2, 2, 2])

# テンソルをリストに変換
original_list = original_tensor.tolist()

# 同じ数字のセットをサブリストに分割し、そのインデックスも取得
grouped_values = []
grouped_indices = []
for value, group in groupby(enumerate(original_list), key=lambda x: x[1]):
    group = list(group)
    grouped_values.append([x[1] for x in group])  # 値
    grouped_indices.append([x[0] for x in group])  # インデックス

# 値のサブリストとインデックスを同じ順番でシャッフル
combined = list(zip(grouped_values, grouped_indices))
random.shuffle(combined)

# シャッフルされたサブリストを結合
shuffled_values = [item for group, _ in combined for item in group]
shuffled_indices = [idx for _, indices in combined for idx in indices]

# リストを再びテンソルに変換
shuffled_tensor = torch.tensor(shuffled_values)

print("もとのテンソル", original_tensor)
print("シャッフルされたテンソル:", shuffled_tensor)
print("シャッフルされたインデックス:", shuffled_indices)
    

もとのテンソル tensor([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 2, 2, 2])
シャッフルされたテンソル: tensor([4, 4, 4, 2, 2, 2, 5, 5, 5, 1, 1, 1, 2, 2, 2, 3, 3, 3])
シャッフルされたインデックス: [9, 10, 11, 15, 16, 17, 12, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8]
